# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 24 2023 2:41PM,261683,20,9413067,"Exact-Rx, Inc.",Released
1,Apr 24 2023 2:41PM,261683,20,9413068,"Exact-Rx, Inc.",Released
2,Apr 24 2023 2:41PM,261683,20,9413069,"Exact-Rx, Inc.",Released
3,Apr 24 2023 2:41PM,261683,20,9413070,"Exact-Rx, Inc.",Released
4,Apr 24 2023 2:41PM,261683,20,9413071,"Exact-Rx, Inc.",Released
5,Apr 24 2023 2:41PM,261683,20,9413072-Hold_Rx(AL),"Exact-Rx, Inc.",Released
6,Apr 24 2023 2:41PM,261683,20,9413073,"Exact-Rx, Inc.",Released
7,Apr 24 2023 2:41PM,261683,20,9413074,"Exact-Rx, Inc.",Released
8,Apr 24 2023 2:41PM,261683,20,9413075-Hold_Rx(IA),"Exact-Rx, Inc.",Released
9,Apr 24 2023 2:41PM,261683,20,9413076,"Exact-Rx, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,261676,Released,13
27,261680,Released,1
28,261681,Released,1
29,261682,Released,5
30,261683,Released,22


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
261676,NaN,5.0,13.0
261680,NaN,NaN,1.0
261681,NaN,NaN,1.0
261682,NaN,NaN,5.0
261683,NaN,NaN,22.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
261618,18.0,0.0,10.0
261621,0.0,0.0,1.0
261622,0.0,0.0,50.0
261635,2.0,6.0,1.0
261636,12.0,0.0,28.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
261618,18,0,10
261621,0,0,1
261622,0,0,50
261635,2,6,1
261636,12,0,28


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,261618,18,0,10
1,261621,0,0,1
2,261622,0,0,50
3,261635,2,6,1
4,261636,12,0,28


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,261618,18,,10
1,261621,,,1
2,261622,,,50
3,261635,2,6,1
4,261636,12,,28


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 24 2023 2:41PM,261683,20,"Exact-Rx, Inc."
22,Apr 24 2023 2:29PM,261682,10,Bio-PRF
27,Apr 24 2023 2:10PM,261681,19,Else Nutrition
28,Apr 24 2023 2:02PM,261680,19,"Emersa Waterbox, LLC"
29,Apr 24 2023 1:31PM,261676,19,"AdvaGen Pharma, Ltd"
47,Apr 24 2023 12:40PM,261664,22,"NBTY Global, Inc."
48,Apr 24 2023 12:38PM,261663,18,"Emersa Waterbox, LLC"
49,Apr 24 2023 12:36PM,261662,18,"Emersa Waterbox, LLC"
50,Apr 24 2023 12:29PM,261659,18,"Emersa Waterbox, LLC"
51,Apr 24 2023 12:29PM,261660,10,"ClearSpec, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Apr 24 2023 2:41PM,261683,20,"Exact-Rx, Inc.",,,22
1,Apr 24 2023 2:29PM,261682,10,Bio-PRF,,,5
2,Apr 24 2023 2:10PM,261681,19,Else Nutrition,,,1
3,Apr 24 2023 2:02PM,261680,19,"Emersa Waterbox, LLC",,,1
4,Apr 24 2023 1:31PM,261676,19,"AdvaGen Pharma, Ltd",,5,13
5,Apr 24 2023 12:40PM,261664,22,"NBTY Global, Inc.",,,1
6,Apr 24 2023 12:38PM,261663,18,"Emersa Waterbox, LLC",,,1
7,Apr 24 2023 12:36PM,261662,18,"Emersa Waterbox, LLC",,,1
8,Apr 24 2023 12:29PM,261659,18,"Emersa Waterbox, LLC",,,1
9,Apr 24 2023 12:29PM,261660,10,"ClearSpec, LLC",,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 24 2023 2:41PM,261683,20,"Exact-Rx, Inc.",22,,
1,Apr 24 2023 2:29PM,261682,10,Bio-PRF,5,,
2,Apr 24 2023 2:10PM,261681,19,Else Nutrition,1,,
3,Apr 24 2023 2:02PM,261680,19,"Emersa Waterbox, LLC",1,,
4,Apr 24 2023 1:31PM,261676,19,"AdvaGen Pharma, Ltd",13,5,
5,Apr 24 2023 12:40PM,261664,22,"NBTY Global, Inc.",1,,
6,Apr 24 2023 12:38PM,261663,18,"Emersa Waterbox, LLC",1,,
7,Apr 24 2023 12:36PM,261662,18,"Emersa Waterbox, LLC",1,,
8,Apr 24 2023 12:29PM,261659,18,"Emersa Waterbox, LLC",1,,
9,Apr 24 2023 12:29PM,261660,10,"ClearSpec, LLC",,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 24 2023 2:41PM,261683,20,"Exact-Rx, Inc.",22,,
1,Apr 24 2023 2:29PM,261682,10,Bio-PRF,5,,
2,Apr 24 2023 2:10PM,261681,19,Else Nutrition,1,,
3,Apr 24 2023 2:02PM,261680,19,"Emersa Waterbox, LLC",1,,
4,Apr 24 2023 1:31PM,261676,19,"AdvaGen Pharma, Ltd",13,5,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 24 2023 2:41PM,261683,20,"Exact-Rx, Inc.",22.0,NaN,NaN
1,Apr 24 2023 2:29PM,261682,10,Bio-PRF,5.0,NaN,NaN
2,Apr 24 2023 2:10PM,261681,19,Else Nutrition,1.0,NaN,NaN
3,Apr 24 2023 2:02PM,261680,19,"Emersa Waterbox, LLC",1.0,NaN,NaN
4,Apr 24 2023 1:31PM,261676,19,"AdvaGen Pharma, Ltd",13.0,5.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 24 2023 2:41PM,261683,20,"Exact-Rx, Inc.",22.0,0.0,0.0
1,Apr 24 2023 2:29PM,261682,10,Bio-PRF,5.0,0.0,0.0
2,Apr 24 2023 2:10PM,261681,19,Else Nutrition,1.0,0.0,0.0
3,Apr 24 2023 2:02PM,261680,19,"Emersa Waterbox, LLC",1.0,0.0,0.0
4,Apr 24 2023 1:31PM,261676,19,"AdvaGen Pharma, Ltd",13.0,5.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1308255,7.0,8.0,2.0
15,784904,39.0,0.0,30.0
18,1308295,5.0,0.0,0.0
19,785037,15.0,5.0,0.0
20,2354820,79.0,0.0,0.0
22,261664,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1308255,7.0,8.0,2.0
1,15,784904,39.0,0.0,30.0
2,18,1308295,5.0,0.0,0.0
3,19,785037,15.0,5.0,0.0
4,20,2354820,79.0,0.0,0.0
5,22,261664,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,7.0,8.0,2.0
1,15,39.0,0.0,30.0
2,18,5.0,0.0,0.0
3,19,15.0,5.0,0.0
4,20,79.0,0.0,0.0
5,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,7.0
1,15,Released,39.0
2,18,Released,5.0
3,19,Released,15.0
4,20,Released,79.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,18,19,20,22
Status,,,,,,
Completed,2.0,30.0,0.0,0.0,0.0,0.0
Executing,8.0,0.0,0.0,5.0,0.0,0.0
Released,7.0,39.0,5.0,15.0,79.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,18,19,20,22
0,Completed,2.0,30.0,0.0,0.0,0.0,0.0
1,Executing,8.0,0.0,0.0,5.0,0.0,0.0
2,Released,7.0,39.0,5.0,15.0,79.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,18,19,20,22
0,Completed,2.0,30.0,0.0,0.0,0.0,0.0
1,Executing,8.0,0.0,0.0,5.0,0.0,0.0
2,Released,7.0,39.0,5.0,15.0,79.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()